# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

boston = datasets.load_boston()

def populate_table(tablename, feature_names):
    X = boston.data  
    y = boston.target
    N = X.shape[0]
    y = y.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : list(boston.feature_names),
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/boston.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/boston.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TGT
230,230,230,0.53700,0.0,6.20,0,0.504,5.981,68.1,3.6715,8,307,17.4,378.35,11.65,24
112,112,112,0.12329,0.0,10.01,0,0.547,5.913,92.9,2.3534,6,432,17.8,394.95,16.21,18
125,125,125,0.16902,0.0,25.65,0,0.581,5.986,88.4,1.9929,2,188,19.1,385.02,14.81,21
9,9,9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18
213,213,213,0.14052,0.0,10.59,0,0.489,6.375,32.3,3.9454,4,277,18.6,385.81,9.38,28
109,109,109,0.26363,0.0,8.56,0,0.520,6.229,91.2,2.5451,5,384,20.9,391.23,15.55,19
127,127,127,0.25915,0.0,21.89,0,0.624,5.693,96.0,1.7883,4,437,21.2,392.11,17.19,16
244,244,244,0.20608,22.0,5.86,0,0.431,5.593,76.5,7.9549,7,330,19.1,372.49,12.50,17
406,406,406,20.71620,0.0,18.10,0,0.659,4.138,100.0,1.1781,24,666,20.2,370.22,23.34,11
490,490,490,0.20746,0.0,27.74,0,0.609,5.093,98.0,1.8226,4,711,20.1,318.43,29.68,8


## Training a Model

In [5]:
from sklearn.random_projection import GaussianRandomProjection

clf = GaussianRandomProjection(n_components = 7)

clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


GaussianRandomProjection(n_components=7)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3", "Feature_4", "Feature_5", "Feature_6", "Feature_7", "Feature_8", "Feature_9", "Feature_10", "Feature_11", "Feature_12" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "random_proj_1", "random_proj_2", "random_proj_3", "random_proj_4", "random_proj_5", "random_proj_6", "random_proj_7" };

		return lOutputs;
	}

	tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3, std::any Feature_4, std::any Feature_5, std::any Feature_6, std::any Feature_7, std::any Feature_8, std::any Feature_9, std::any Feature_10, std::any Feature_11, std::any Feature_12) {

		tTable lTable;

		lTable["random_proj_1"] = { Feature_0 * -0.3371621336685478 + Feature_1 * 0.16376276828690237 + Feature_2 * -0.5933688040542326 + Feature_3 * -0.00055262515067

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:100])
        print(result2[-100:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/boston2.csv" , ["Feature_" + str(i) for i,x in enumerate(metadata["features"])])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/boston2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140219421484224.exe /tmp/sklearn2sql_cpp_140219421484224.cpp'
idx,random_proj_1,random_proj_2,random_proj_3,random_proj_4,random_proj_5,random_proj_6,random_proj_
676595,2.59884097546940,-40.43804490498381,340.89389348135376,-115.56572564718974,24.16281053826525



In [10]:
cpp_output.sample(12, random_state=1960)

,idx,random_proj_1,random_proj_2,random_proj_3,random_proj_4,random_proj_5,random_proj_6,random_proj_7
230,230,-63.968561,121.753079,16.079952,-52.561204,363.864157,-94.224541,40.649887
112,112,-118.128680,67.992216,49.081401,-73.595047,447.671385,-57.164375,103.952791
125,125,-53.351673,200.014474,-19.552038,-19.828260,274.937110,-142.167448,-3.616306
9,9,-75.611064,131.163853,13.511403,-53.012080,363.923051,-96.525931,47.549664
213,213,-29.866380,143.416431,3.997075,-42.083088,358.437415,-100.931889,15.328014
109,109,-103.535829,91.061225,37.991335,-65.738070,411.629979,-73.136504,80.067524
127,127,-130.492038,64.495459,50.498865,-67.040316,447.006416,-49.831520,107.987065
244,244,-75.637643,103.794221,18.846167,-56.661779,375.350358,-75.007249,47.351784
406,406,-212.205830,-73.063176,128.187797,-121.528405,599.529202,38.641657,200.090467
490,490,-237.039572,-138.697514,144.863556,-115.142725,598.410568,97.961241,261.241424


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)


,idx,random_proj_1,random_proj_2,random_proj_3,random_proj_4,random_proj_5,random_proj_6,random_proj_7
230,230,-63.968561,121.753079,16.079952,-52.561204,363.864157,-94.224541,40.649887
112,112,-118.128680,67.992216,49.081401,-73.595047,447.671385,-57.164375,103.952791
125,125,-53.351673,200.014474,-19.552038,-19.828260,274.937110,-142.167448,-3.616306
9,9,-75.611064,131.163853,13.511403,-53.012080,363.923051,-96.525931,47.549664
213,213,-29.866380,143.416431,3.997075,-42.083088,358.437415,-100.931889,15.328014
109,109,-103.535829,91.061225,37.991335,-65.738070,411.629979,-73.136504,80.067524
127,127,-130.492038,64.495459,50.498865,-67.040316,447.006416,-49.831520,107.987065
244,244,-75.637643,103.794221,18.846167,-56.661779,375.350358,-75.007249,47.351784
406,406,-212.205830,-73.063176,128.187797,-121.528405,599.529202,38.641657,200.090467
490,490,-237.039572,-138.697514,144.863556,-115.142725,598.410568,97.961241,261.241424


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,random_proj_1_skl,random_proj_2_skl,random_proj_3_skl,random_proj_4_skl,random_proj_5_skl,random_proj_6_skl,random_proj_7_skl,idx_cpp,random_proj_1_cpp,random_proj_2_cpp,random_proj_3_cpp,random_proj_4_cpp,random_proj_5_cpp,random_proj_6_cpp,random_proj_7_cpp
230,230,-63.968561,121.753079,16.079952,-52.561204,363.864157,-94.224541,40.649887,230,-63.968561,121.753079,16.079952,-52.561204,363.864157,-94.224541,40.649887
112,112,-118.128680,67.992216,49.081401,-73.595047,447.671385,-57.164375,103.952791,112,-118.128680,67.992216,49.081401,-73.595047,447.671385,-57.164375,103.952791
125,125,-53.351673,200.014474,-19.552038,-19.828260,274.937110,-142.167448,-3.616306,125,-53.351673,200.014474,-19.552038,-19.828260,274.937110,-142.167448,-3.616306
9,9,-75.611064,131.163853,13.511403,-53.012080,363.923051,-96.525931,47.549664,9,-75.611064,131.163853,13.511403,-53.012080,363.923051,-96.525931,47.549664
213,213,-29.866380,143.416431,3.997075,-42.083088,358.437415,-100.931889,15.328014,213,-29.866380,143.416431,3.997075,-42.083088,358.437415,-100.931889,15.328014
109,109,-103.535829,91.061225,37.991335,-65.738070,411.629979,-73.136504,80.067524,109,-103.535829,91.061225,37.991335,-65.738070,411.629979,-73.136504,80.067524
127,127,-130.492038,64.495459,50.498865,-67.040316,447.006416,-49.831520,107.987065,127,-130.492038,64.495459,50.498865,-67.040316,447.006416,-49.831520,107.987065
244,244,-75.637643,103.794221,18.846167,-56.661779,375.350358,-75.007249,47.351784,244,-75.637643,103.794221,18.846167,-56.661779,375.350358,-75.007249,47.351784
406,406,-212.205830,-73.063176,128.187797,-121.528405,599.529202,38.641657,200.090467,406,-212.205830,-73.063176,128.187797,-121.528405,599.529202,38.641657,200.090467
490,490,-237.039572,-138.697514,144.863556,-115.142725,598.410568,97.961241,261.241424,490,-237.039572,-138.697514,144.863556,-115.142725,598.410568,97.961241,261.241424


In [14]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())
    

count    506.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    5.060000e+02
mean    -1.021142e-15
std      1.472642e-14
min     -5.684342e-14
25%     -4.163336e-16
50%      0.000000e+00
75%      0.000000e+00
max      5.684342e-14
dtype: float64
count    5.060000e+02
mean    -2.057204e-15
std      1.479493e-14
min     -5.684342e-14
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      5.684342e-14
dtype: float64
count    5.060000e+02
mean    -1.261617e-16
std      7.708536e-15
min     -5.684342e-14
25%     -1.776357e-15
50%      0.000000e+00
75%      1.776357e-15
max      5.684342e-14
dtype: float64
count    5.060000e+02
mean    -1.825505e-16
std      7.159155e-15
min     -2.842171e-14
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.842171e-14
dtype: float64
count    5.060000e+02
mean     1.235726e-15
std      4.764336e-14
min     -1.136868e-13
25%      0.00